# Interpolation

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from itertools import combinations

# from scipy.spatial.distance import pdist
# from scipy.stats import chisquare, chi2
from scipy.stats import ttest_1samp, t

from pykrige.ok import OrdinaryKriging
from pykrige.rk import Krige
import pykrige.kriging_tools as kt

from sklearn.model_selection import cross_val_predict

from joblib import Parallel, delayed
import multiprocessing

In [ ]:
import prepostprocessing.pre_processing as preproc
import prepostprocessing.post_processing as postproc
import nugget_estimation.kriging_tools as krigingtools
import nugget_estimation.detrending as detrending
import plots.plots as plots
import statistics.ttest as ttest
import prepostprocessing.cleaning as cleaning
import cross_validation.leave_one_out_cross_validation as loocv
import qapf.qapf as qapf
import statistics.simulations as sim

import toolbox.toolbox as tb

In [ ]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readtheMLcs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

## EDA

## Critical distance estimation

## Classification map

## Detrending

## Variogram modelling

* Variogram model (str)
* Sill (float)
* Range (float)
* Nugget (float)
* Number of lag bins (int)
* Weighted (Bool)

## Leave One Out Cross Validation (LOOCV) of PCA

## Calculating predictions for best model

## Calculate clr solutions

## Error budget

## Contour plots

## Biplots

## Simulations